In [13]:
originData = sc.textFile('douban-movie-comments.csv')

In [14]:
originData.count()

5989

In [15]:
originDistinctData = originData.distinct()

In [16]:
originDistinctData.count()

5739

In [36]:
rateDocument = originDistinctData.map(lambda line: line.split(',')).filter(lambda line: len(line) == 2)

In [37]:
rateDocument.count()

5463

In [38]:
rateDocument.take(3)

[[u'4.0',
  u' \u539f\u4f5c\u91cc\u7684\u4e00\u4e9b\u8d85\u7ea7\u7ecf\u5178\u7684\u6897\uff0c\u5728\u8fd9\u90e8\u7eed\u96c6\u91cc\u90fd\u88ab\u7528\u4e0a\u4e86\u3002\u6bd4\u5982\u90a3\u4e2a\u5403\u4e86\u7ea2\u85af\u540e\u653e\u5927\u5c41\u7684\u6897\uff0c\u6765\u5f97\u771f\u591f\u6b8b\u66b4\u7684\u3002'],
 [u'1.0',
  u' \u5b8c\u5168\u6ca1\u6709\u8bda\u610f\u7684\u6b3a\u9a97\uff0c0\u5267\u60c50\u6f14\u6280\uff0c\u5982\u679c\u4e0d\u662f\u65f6\u95f4\u5f88\u591a\uff0c\u5343\u4e07\u522b\u6d6a\u8d39\u4e86'],
 [u'1.0', u' \u51b7\u996d\uff0c\u5168\u90fd\u662f\u51b7\u996d\u3002']]

In [55]:
fiveRateDocument = rateDocument.filter(lambda line: line[0] == '5.0') 
fourRateDocument = rateDocument.filter(lambda line: line[0] == '4.0')
threeRateDocument = rateDocument.filter(lambda line: line[0] == '3.0')
twoRateDocument = rateDocument.filter(lambda line: line[0] == '2.0')
oneRateDocument = rateDocument.filter(lambda line: line[0] == '1.0')

In [57]:
print(fiveRateDocument.count())
print(fourRateDocument.count())
print(threeRateDocument.count())
print(twoRateDocument.count())
print(oneRateDocument.count())

920
552
509
602
2687


In [74]:
posRateDocument = fourRateDocument.union(fiveRateDocument)
negRateDocument = (oneRateDocument.union(twoRateDocument).union(threeRateDocument))

print(posRateDocument.count())
print(negRateDocument.count())

1472
3798


In [75]:
negRateDocument = sc.parallelize(negRateDocument.take(posRateDocument.count()))

In [76]:
print(posRateDocument.count())
print(negRateDocument.count())

1472
1472


In [77]:
posRateDocument.repartition(1)
negRateDocument.repartition(1)

MapPartitionsRDD[141] at repartition at NativeMethodAccessorImpl.java:-2

In [104]:
allRateDocument = posRateDocument.union(negRateDocument)
allRateDocument.repartition(1)
rate = allRateDocument.map(lambda s: s[0])
document = allRateDocument.map(lambda s: s[1])

In [106]:
import jieba

words = document.map(lambda w: "/".join(jieba.cut_for_search(w))).map(lambda line: line.split("/"))

In [107]:
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(words)
tf.cache()

PythonRDD[195] at RDD at PythonRDD.scala:48

In [108]:
idfModel = IDF().fit(tf)
tfidf = idfModel.transform(tf)

In [112]:
from pyspark.mllib.regression import LabeledPoint

zipped = rate.zip(tfidf)
data = zipped.map(lambda line: LabeledPoint(line[0], line[1]))
training,test = data.randomSplit([0.6, 0.4], seed=0)

In [113]:
from pyspark.mllib.classification import NaiveBayes

NBmodel = NaiveBayes.train(training, 1.0)
predictionAndLabel = test.map(lambda p: (NBmodel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda x: 1.0 if x[0] == x[1] else 0.0).count()

In [139]:
# yourDocument = input("Please input your comment:")
yourWords = "/".join(jieba.cut_for_search("well good")).split("/")
yourtf = hashingTF.transform(yourWords)
yourtfidf = idfModel.transform(yourtf)
print("NaiveBayes Model Predict: %d" % NBmodel.predict(yourtfidf))

NaiveBayes Model Predict: 5
